It is recommended using the docker container for OpenVINO.

```sh
docker pull openvino/ubuntu20_runtime
docker run -it --rm openvino/ubuntu20_runtime bash
```

Install the packages first.

In [ ]:
!pip install --no-cache-dir numpy opencv-python matplotlib

In [ ]:
import logging
import time
import os
import numpy as np
import cv2
import codecs
import matplotlib.pyplot as plt
from openvino.inference_engine import IECore

# Device information

Use `lscpu` to check whether the CPU is available. Use `lsusb` to check whether the supported USB plugins are available, like Movidius NCS, etc.

In [ ]:
!lscpu

In [ ]:
!lsusb

# Person Detection

## Dataset Preparation

Find the images under the directory and prepare their paths.

In [ ]:
imgFolderPath = "./pd/images"

In [ ]:
allImages = next(os.walk(imgFolderPath))[2]
allImages = [img for img in allImages if img[-3:] == "jpg"]
print("There are {} images.".format(len(allImages)))

allImagePaths = [os.path.join(imgFolderPath, imgName) for imgName in allImages]
print("The first image path: {}".format(allImagePaths[0]))

Show an example image.

In [ ]:
img = cv2.imread(allImagePaths[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

## OpenVINO Inference Runtime

Before inference, you have to convert the model corresponding to version of OpenVINO.

In [ ]:
# plugin initialization
ie = IECore()

myriad_config = {"VPU_HW_STAGES_OPTIMIZATION": "YES"}
# ie.set_config(myriad_config, "MYRIAD")

# read the intermediate representations
net = ie.read_network("./pd/model/pd.xml", 
                      "./pd/model/pd.bin")

# prepare the I/Os
net.batch_size = 1
net_inputs = net.inputs
net_outputs = net.outputs
inputNames = list(net_inputs.keys())
outputNames = list(net_outputs.keys())
print("Network input:  {}".format(inputNames))
print("Network output: {}".format(outputNames))

input_blob = next(iter(net.inputs))
output_blob = next(iter(net.outputs))

# read and preprocess input images
# in openvino, the shape is (batch_size, channels, height, width)
_, _, height, width = net.inputs[input_blob].shape
print("Image height: {} and width: {}".format(height, width))

Preprocess the image as a batch. Unlike the other platforms, the batch shape for OpenVINO is `[B, C, H, W]`.

In [ ]:
def preprocessing(image, plot=False):
  """Preprocess the image as a batch."""
  image = cv2.resize(image, (width, height))
  if plot:
    plt.imshow(image)
    plt.show()
  image = np.swapaxes(image, 2, 0)
  image = np.swapaxes(image, 1, 2)
  image = np.expand_dims(image, axis=0)    
  assert image.ndim == 4
  return image

processedImage = preprocessing(img, plot=True)
print("The processed image's shape: {}.".format(processedImage.shape))

Load the model and infer the preprocessed image by the blob name.

In [ ]:
# loading the model to the plugin
model = ie.load_network(network=net, device_name="CPU")

# start the inference
res = model.infer(inputs={input_blob: processedImage})

# processing the outputs
outputs = res[output_blob][0]
print("Output: {}".format(outputs.shape))

Bundle all of the above and test the inference speed.

In [ ]:
ttlTime = 0.0
count = 0
for imgPath in allImagePaths:
  img = cv2.imread(imgPath)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
  processedImage = preprocessing(img)
  startTime = time.perf_counter()
  _ = model.infer(inputs={input_blob: processedImage})
  ttlTime += time.perf_counter() - startTime
  count += 1
    
print("Total Time: {:.4f} secs for {} images.".format(ttlTime, count))
print("Inference Time per Image: {:.4f} secs.".format(ttlTime / count))
print("FPS: {:.4f}.".format(count / ttlTime))